In [28]:
import numpy as np
from scipy.fft import fft
import pandas as pd
from scipy.linalg import orth
from typing import Tuple
from tqdm import tqdm
import time


In [29]:
# The class to generate the matrices.
class MatrixGeneration:

  """
    The MatrixGeneration class takes three arguments in the constructor.

    m: The number of rows.
    n: The number of cols.

  """
  def __init__(self,m,n,kappa):
    self.m=m
    self.n=n
    self.kappa=kappa

    ## TODO: Some places we return self.A, some places we return A.lets make it uniform.
    #matrices with good condition number
  def ug_matrix(self):
    U = orth(np.random.rand(self.m, self.n))   # m x n orthonormal matrix
    V = orth(np.random.rand(self.n, self.n))   # n x n orthonormal matrix
    S = np.diag(np.linspace(1, 1/self.kappa, self.n))  # Diagonal scaling matrix
    self.A = U @ S @ V   # Final matrix
    return self.A

  #nb and ng matrices are the matrices which have a bad leverage score
  def nb_matrix(self):
      # Determine dimensions for constructing the matrix
      half_num_cols = int(self.n / 2)
      top_block_rows = self.m - half_num_cols

      # Random Gaussian block
      gaussian_block = np.random.normal(0, 1, (top_block_rows, half_num_cols))

      # Tiny noise block
      tiny_noise_block = 1e-8 * np.random.rand(top_block_rows, half_num_cols)

      # Identity matrix for the stable part
      identity_block = np.identity(half_num_cols)

      # Construct matrix blocks
      top_left = self.kappa * gaussian_block
      top_right = tiny_noise_block
      bottom_left = np.zeros((half_num_cols, top_block_rows))
      bottom_right = identity_block

      # Assemble full matrix
      top_half = np.hstack([top_left, top_right])
      bottom_half = np.hstack([bottom_left[:, :half_num_cols], bottom_right])
      A = np.vstack([top_half, bottom_half])

      return A


In [30]:
import numpy as np
from typing import Tuple

class MatrixApproximationLeverage:
    """
    Performs approximate matrix multiplication using leverage score sampling techniques.
    """

    @staticmethod
    def leverage_score_sampling(A: np.ndarray, B: np.ndarray, samples: int) -> Tuple[np.ndarray, np.ndarray]:
        """
        Selects rows from A and B based on leverage scores to approximate A @ B.T.

        Args:
            A (np.ndarray): Input matrix of shape (m, n)
            B (np.ndarray): Input matrix of shape (n, p)
            samples (int): Number of rows (and singular vectors) to use

        Returns:
            Tuple[np.ndarray, np.ndarray]: Scaled sampled matrices A' and B'
        """
        m, _ = A.shape
        n, _ = B.shape

        # 1) full SVD, then truncate to top-k singular vectors
        U, _, _  = np.linalg.svd(A, full_matrices=False)
        Ub, _, _ = np.linalg.svd(B, full_matrices=False)

        Uk  = U[:, :samples]   # (m × k)
        Ubk = Ub[:, :samples]  # (n × k)

        # 2) leverage scores = row-norms² of those truncated bases
        levA = np.sum(Uk**2,  axis=1)  # length-m
        levB = np.sum(Ubk**2, axis=1)  # length-n

        # 3) normalize to get true probability distributions
        pA = levA / levA.sum()
        pB = levB / levB.sum()

        # 4) random sample exactly k rows according to pA, pB
        idxA = np.random.choice(m, size=samples, replace=False, p=pA)
        idxB = np.random.choice(n, size=samples, replace=False, p=pB)

        # 5) build and scale
        sampled_A = A[idxA, :]
        sampled_B = B[idxB, :]

        scaled_A = sampled_A * np.sqrt(m / samples)
        scaled_B = sampled_B * np.sqrt(n / samples)

        return scaled_A, scaled_B

    @staticmethod
    def sqrt_leverage_score_sampling(A: np.ndarray, B: np.ndarray, samples: int) -> Tuple[np.ndarray, np.ndarray]:
        """
        Selects rows using the square root of leverage scores.

        Args:
            A (np.ndarray): Input matrix of shape (m, n)
            B (np.ndarray): Input matrix of shape (n, p)
            samples (int): Number of rows (and singular vectors) to use

        Returns:
            Tuple[np.ndarray, np.ndarray]: Scaled sampled matrices A' and B'
        """
        m, _ = A.shape
        n, _ = B.shape

        U, _, _  = np.linalg.svd(A, full_matrices=False)
        Ub, _, _ = np.linalg.svd(B, full_matrices=False)

        Uk  = U[:, :samples]
        Ubk = Ub[:, :samples]

        # 1) “sqrt-leverage” = √(sum of squares)
        levA = np.sqrt(np.sum(Uk**2,  axis=1))
        levB = np.sqrt(np.sum(Ubk**2, axis=1))

        # 2) normalize to get true probability distributions
        pA = levA / levA.sum()
        pB = levB / levB.sum()

        # 3) random sample exactly k rows according to pA, pB
        idxA = np.random.choice(m, size=samples, replace=False, p=pA)
        idxB = np.random.choice(n, size=samples, replace=False, p=pB)

        sampled_A = A[idxA, :]
        sampled_B = B[idxB, :]

        scaled_A = sampled_A * np.sqrt(m / samples)
        scaled_B = sampled_B * np.sqrt(n / samples)

        return scaled_A, scaled_B


In [31]:
import numpy as np
from typing import Tuple, Dict, Any

class MatrixApproximationSampling:
    def __init__(self, hash_size: int = 10000, seed: int = 42):
        """
        Initializes the sampler with a random hash vector used for priority and threshold sampling.
        """
        np.random.seed(seed)
        self.hash = np.random.uniform(0, 1, size=hash_size)

    def uniform_sampling(self, A: np.ndarray, B: np.ndarray, num_samples: int) -> Tuple[np.ndarray, np.ndarray]:
        """
        Uniformly samples rows from A and B to approximate A.T @ B.

        For matrices A and B of shape (n, d), this method selects 'num_samples' rows uniformly at random,
        scales them appropriately, and returns two matrices C and R such that C @ R approximates A.T @ B.
        
        C has shape (d, num_samples) and R has shape (num_samples, d), so the product C @ R is (d, d).
        """
        # A and B must both be of shape (n, d)
        n, d = A.shape
        if B.shape != (n, d):
            raise ValueError("A and B must have the same shape.")
        
        # Initialize matrices: we store rows of A (transposed) in C and rows of B in R.
        C = np.zeros((d, num_samples))
        R = np.zeros((num_samples, d))
        
        # Compute scaling factor so the estimator is unbiased.
        scaling = np.sqrt(n / num_samples)
        
        for t in range(num_samples):
            idx = np.random.choice(n)  # sample uniformly from 0 to n-1
            # Each column of C is the scaled sampled row of A (transposed).
            C[:, t] = A[idx, :] * scaling
            # Each row of R is the scaled sampled row of B.
            R[t, :] = B[idx, :] * scaling
        
        return C, R

    def priority_sampling(self, A: np.ndarray, num_samples: int) -> Dict[str, Any]:
        """
        Performs priority sampling on the rows of matrix A.

        Returns a sketch with selected row indices, row values, and a threshold.
        """
        n, d = A.shape
        r = np.full(n, np.inf)

        for i in range(n):
            norm_sq = np.linalg.norm(A[i]) ** 2
            if norm_sq > 0:
                r[i] = self.hash[i] / norm_sq

        threshold = np.partition(r, num_samples)[num_samples] if num_samples < n else np.inf

        selected_indices = np.where(r < threshold)[0]
        selected_rows = A[selected_indices]

        return {
            'indices': selected_indices,
            'values': selected_rows,
            'threshold': threshold
        }

    def threshold_sampling(self, A: np.ndarray, num_samples: int) -> Dict[str, Any]:
        """
        Performs threshold sampling based on row norms and a hash function.

        Returns a sketch with selected row indices, row values, and a threshold value.
        """
        n, d = A.shape
        A_norm_sq = np.linalg.norm(A, 'fro') ** 2
        tau = num_samples / A_norm_sq if A_norm_sq > 0 else float('inf')

        selected_indices = []
        selected_rows = []

        for i in range(n):
            row_norm_sq = np.linalg.norm(A[i]) ** 2
            if self.hash[i] <= tau * row_norm_sq:
                selected_indices.append(i)
                selected_rows.append(A[i])

        return {
            'indices': np.array(selected_indices),
            'values': np.array(selected_rows),
            'threshold': tau
        }

    @staticmethod
    def approximate_matrix_multiplication(sketch_A: Dict[str, Any], sketch_B: Dict[str, Any]) -> np.ndarray:
        """
        Approximates matrix multiplication using row sketches.
        Only uses rows that are common to both A and B sketches.
        """
        indices_A = sketch_A['indices']
        indices_B = sketch_B['indices']
        common_indices = np.intersect1d(indices_A, indices_B)

        if common_indices.size == 0:
            return np.zeros((sketch_A['values'].shape[1], sketch_B['values'].shape[1]))

        result = np.zeros((sketch_A['values'].shape[1], sketch_B['values'].shape[1]))

        for idx in common_indices:
            i_A = np.where(indices_A == idx)[0][0]
            i_B = np.where(indices_B == idx)[0][0]

            row_A = sketch_A['values'][i_A]
            row_B = sketch_B['values'][i_B]

            norm_A = np.linalg.norm(row_A) ** 2
            norm_B = np.linalg.norm(row_B) ** 2

            # Multiply by thresholds instead of dividing
            denom = min(1.0, norm_A * sketch_A['threshold'], norm_B * sketch_B['threshold'])

            if denom > 0:
                result += np.outer(row_A, row_B) / denom

        return result


In [32]:
import numpy as np
import math
from typing import Dict

class JLemma:
    """
    Implements various Johnson-Lindenstrauss (JL) transform sketches:
      - Gaussian JL
      - PHD JL (using fast in-place FWHT)
      - Count-Sketch JL
    """
    def __init__(self, seed: int = None):
        """
        seed: RNG seed for reproducibility
        """
        self.rng = np.random.default_rng(seed)

    def gaussian_JL(self, A: np.ndarray, B: np.ndarray, samples: int) -> np.ndarray:
        """
        Gaussian JL: Dense random projection to approximate A.T @ B.
        """
        m, _ = A.shape
        S = self.rng.normal(0, 1/np.sqrt(samples), size=(m, samples))
        A_proj = A.T @ S    # (n x samples)
        B_proj = B.T @ S    # (p x samples)
        return A_proj @ B_proj.T  # (n x p)

    @staticmethod
    def _hadamard_transform(X: np.ndarray) -> np.ndarray:
        """
        In-place Fast Walsh–Hadamard Transform (FWHT) on each column of X.
        Assumes X.shape[0] is a power of 2.
        """
        M, p = X.shape
        H = X.copy()
        h = 1
        while h < M:
            for i in range(0, M, 2*h):
                top = H[i:i+h, :]
                bot = H[i+h:i+2*h, :]
                H[i:i+h, :] = top + bot
                H[i+h:i+2*h, :] = top - bot
            h *= 2
        return H

    def PHD_JL(self, A: np.ndarray, B: np.ndarray, samples: int) -> np.ndarray:
        """
        PHD JL: Fast JL using randomized Hadamard + subsampling.
        Approximates A.T @ B by (S H D A)^T (S H D B).
        samples: sketch dimension t (with replacement sampling).
        """
        n, p1 = A.shape
        _, p2 = B.shape
        t = samples
        # Pad to power of two
        mprime = 1 << ((n - 1).bit_length())
        if mprime > n:
            A_pad = np.zeros((mprime, p1), dtype=A.dtype)
            B_pad = np.zeros((mprime, p2), dtype=B.dtype)
            A_pad[:n, :] = A
            B_pad[:n, :] = B
        else:
            A_pad = A.copy()
            B_pad = B.copy()
        # Random sign flip D
        signs = self.rng.choice([-1, 1], size=mprime)
        A_signed = A_pad * signs[:, None]
        B_signed = B_pad * signs[:, None]
        # Hadamard transform H
        A_h = JLemma._hadamard_transform(A_signed)  # (mprime x p1)
        B_h = JLemma._hadamard_transform(B_signed)  # (mprime x p2)
        # Normalize by sqrt(m' * t)
        norm = 1.0 / math.sqrt(mprime * t)
        A_h *= norm
        B_h *= norm
        # Sample t rows with replacement
        idx = self.rng.integers(0, mprime, size=t)
        A_s = A_h[idx, :]  # (t x p1)
        B_s = B_h[idx, :]  # (t x p2)
        # Approximate product
        return A_s.T @ B_s  # (p1 x p2)

    def countsketch_JL(self, A: np.ndarray, B: np.ndarray, s: int) -> np.ndarray:
        """
        Count-Sketch JL: Sparse projection via hashing and sign-flip.
        """
        m, p1 = A.shape
        _, p2 = B.shape
        h = self.rng.integers(0, s, size=m)
        sigma = self.rng.choice([-1, 1], size=m)
        A_s = np.zeros((p1, s), dtype=A.dtype)
        B_s = np.zeros((p2, s), dtype=B.dtype)
        for i in range(m):
            b = h[i]
            A_s[:, b] += sigma[i] * A[i, :]
            B_s[:, b] += sigma[i] * B[i, :]
        return A_s @ B_s.T  # (p1 x p2)



In [33]:
#all the four matrices
"""
There are two key properties being considered here:

1. Leverage Score Distribution
Uniform leverage scores → all rows are equally “important”

Non-uniform leverage scores → some rows are more “important” than others

2. Condition Number (κ / kappa)
Good condition number (low κ) → matrix is stable for computation

Bad condition number (high κ) → matrix is numerically unstable, more sensitive to error

"""

# Case 1: Good Condition Number (kappa = 1e-8)
good_condition = MatrixGeneration(10000, 100, 1e-8)
A_uniform_good = good_condition.ug_matrix()
B_uniform_good = good_condition.ug_matrix()
A_nonuniform_good = good_condition.nb_matrix()
B_nonuniform_good = good_condition.nb_matrix()

# Case 2: Bad Condition Number (kappa = 10)
bad_condition = MatrixGeneration(10000, 100, 10)
A_uniform_bad = bad_condition.ug_matrix()
B_uniform_bad = bad_condition.ug_matrix()
A_nonuniform_bad = bad_condition.nb_matrix()
B_nonuniform_bad = bad_condition.nb_matrix()


In [34]:
sample_sizes = [250, 500, 750, 1000, 1250, 1500, 1750, 2000]
iterations = 5

matrix_types = [
    ("Uniform Good", 1e-8, "ug_matrix"),
    ("Uniform Bad", 10, "ug_matrix"),
    ("Non-uniform Good", 1e-8, "nb_matrix"),
    ("Non-uniform Bad", 10, "nb_matrix"),
]


In [35]:
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
results = []

# Assume matrix_types, sample_sizes, iterations, MatrixGeneration,
# MatrixApproximationLeverage, MatrixApproximationSampling, and JLemma 
# are defined elsewhere in your notebook.

for matrix_name, kappa, gen_method in tqdm(matrix_types, desc="Matrix Types"):
    generator = MatrixGeneration(10000, 100, kappa)
    A = getattr(generator, gen_method)()
    B = getattr(generator, gen_method)()

    A_frob = np.linalg.norm(A, 'fro')
    B_frob = np.linalg.norm(B, 'fro')
    original_product = A.T @ B

    for size in sample_sizes:
        for i in range(iterations):
            
            def add_row(method_name, approx_matrix, duration):
                # Fix: Compute relative error as norm(diff) / (A_frob * B_frob)
                diff = approx_matrix - original_product
                rel_error = np.linalg.norm(diff, 'fro') / (A_frob * B_frob)
                results.append({
                    "matrix": matrix_name,
                    "sample_size": size,
                    "iteration": i + 1,
                    "method": method_name,
                    "time": duration,
                    "frobenius_norm": np.linalg.norm(approx_matrix, 'fro'),
                    "relative_error": rel_error
                })
            
            # Leverage Sampling
            start = time.time()
            S_a, S_b = MatrixApproximationLeverage.leverage_score_sampling(A, B, size)
            end = time.time()
            add_row("Leverage Sampling", S_a.T @ S_b, end - start)
            
            # Sqrt Leverage Sampling
            start = time.time()
            S_a, S_b = MatrixApproximationLeverage.sqrt_leverage_score_sampling(A, B, size)
            end = time.time()
            add_row("Sqrt Leverage Sampling", S_a.T @ S_b, end - start)
            
            # Priority Sampling
            sampler = MatrixApproximationSampling()
            start = time.time()
            S_a = sampler.priority_sampling(A, size)
            S_b = sampler.priority_sampling(B, size)
            approx = sampler.approximate_matrix_multiplication(S_a, S_b)
            end = time.time()
            add_row("Priority Sampling", approx, end - start)
            
            # Threshold Sampling
            start = time.time()
            S_a = sampler.threshold_sampling(A, size)
            S_b = sampler.threshold_sampling(B, size)
            approx = sampler.approximate_matrix_multiplication(S_a, S_b)
            end = time.time()
            add_row("Threshold Sampling", approx, end - start)
            
            # Uniform Sampling
            start = time.time()
            S_a, S_b = sampler.uniform_sampling(A, B, size)
            end = time.time()
            add_row("Uniform Sampling", S_a @ S_b, end - start)
            
            # Gaussian JL
            lemma = JLemma()
            start = time.time()
            G = lemma.gaussian_JL(A, B, size)
            end = time.time()
            add_row("Gaussian JL", G, end - start)
            
            # PHD JL with Hadamard
            start = time.time()
            P = lemma.PHD_JL(A, B, size)  # Ensure your implementation supports Hadamard mode
            end = time.time()
            add_row("PHD JL (Hadamard)", P, end - start)
            
            # CountSketch JL
            start = time.time()
            C = lemma.countsketch_JL(A, B, size)
            end = time.time()
            add_row("CountSketch JL", C, end - start)

# Save the final results to "paper_implementation.csv"
df = pd.DataFrame(results)
df.to_csv("paper_implementation.csv", index=False)
print("Results saved to paper_implementation.csv")

Matrix Types: 100%|██████████| 4/4 [02:16<00:00, 34.03s/it]

Results saved to paper_implementation.csv


In [36]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file
df = pd.read_csv("paper_implementation.csv")

# Ensure sample_size is numeric
df["sample_size"] = pd.to_numeric(df["sample_size"], errors="coerce")

# Group by matrix, sample_size, and method to compute mean and std for relative error
grouped = (
    df.groupby(["matrix", "sample_size", "method"], as_index=False)
      .agg(mean_error=("relative_error", "mean"),
           std_error=("relative_error", "std"),
           mean_time=("time", "mean"),
           std_time=("time", "std"))
)

print("Grouped Data Preview:")
print(grouped.head())

# Set Seaborn style
sns.set_style("whitegrid")
output_dir = "plots"
os.makedirs(output_dir, exist_ok=True)

# Generate and save plots for relative error
unique_matrices = grouped["matrix"].unique()

for mat_name in unique_matrices:
    matrix_df = grouped[grouped["matrix"] == mat_name].copy()
    if matrix_df.empty:
        print(f"No data found for matrix: {mat_name}")
        continue

    matrix_df.sort_values(by="sample_size", inplace=True)
    
    plt.figure(figsize=(12, 8))
    ax = sns.lineplot(
        data=matrix_df,
        x="sample_size",
        y="mean_error",
        hue="method",
        marker="o",
        markersize=8,
        linewidth=2
    )
    
    # Add error bars manually
    for method in matrix_df["method"].unique():
        sub_df = matrix_df[matrix_df["method"] == method]
        plt.errorbar(
            sub_df["sample_size"],
            sub_df["mean_error"],
            yerr=sub_df["std_error"],
            fmt="none",
            ecolor="gray",
            elinewidth=1.5,
            capsize=3
        )
    
    plt.title(f"Relative Error vs. Sample Size for Matrix: {mat_name}", fontsize=16)
    plt.xlabel("Sample Size", fontsize=14)
    plt.ylabel("Relative Error", fontsize=14)
    plt.ylim(bottom=0)
    plt.xticks(rotation=45)
    plt.legend(title="Method")
    plt.tight_layout()
    
    # Save the plot
    filename = f"matrix_{mat_name.replace(' ', '_')}_error_plot.png"
    plt.savefig(os.path.join(output_dir, filename), dpi=300)
    plt.close()
    plt.show()

print(f"Relative error plots saved to '{output_dir}/'")


Grouped Data Preview:
            matrix  sample_size             method  mean_error     std_error  \
0  Non-uniform Bad          250     CountSketch JL    0.063548  1.077469e-03   
1  Non-uniform Bad          250        Gaussian JL    0.063728  1.385415e-03   
2  Non-uniform Bad          250  Leverage Sampling    0.053535  4.544510e-04   
3  Non-uniform Bad          250  PHD JL (Hadamard)    0.010157  2.048497e-09   
4  Non-uniform Bad          250  Priority Sampling    0.062410  0.000000e+00   

   mean_time  std_time  
0   0.052026  0.010837  
1   0.058152  0.004801  
2   0.141318  0.043229  
3   0.160593  0.003449  
4   0.054985  0.007981  
Relative error plots saved to 'plots/'


In [37]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Read the CSV file
df = pd.read_csv("paper_implementation.csv")

# Ensure sample_size is numeric
df["sample_size"] = pd.to_numeric(df["sample_size"], errors="coerce")

# Group by matrix, sample_size, and method to compute mean and std for time
grouped = (
    df.groupby(["matrix", "sample_size", "method"], as_index=False)
      .agg(mean_time=("time", "mean"),
           std_time=("time", "std"))
)

# Set a clean style
sns.set_style("whitegrid")
output_dir = "plots"
os.makedirs(output_dir, exist_ok=True)

# Generate and save plots for computation time
unique_matrices = grouped["matrix"].unique()

for mat_name in unique_matrices:
    matrix_df = grouped[grouped["matrix"] == mat_name].copy()
    if matrix_df.empty:
        print(f"No data found for matrix: {mat_name}")
        continue
    
    matrix_df.sort_values(by="sample_size", inplace=True)
    
    plt.figure(figsize=(12, 8))
    ax = sns.lineplot(
        data=matrix_df,
        x="sample_size",
        y="mean_time",
        hue="method",
        marker="o",
        markersize=8,
        linewidth=2
    )
    
    # Add error bars for time
    for method in matrix_df["method"].unique():
        sub_df = matrix_df[matrix_df["method"] == method]
        plt.errorbar(
            sub_df["sample_size"],
            sub_df["mean_time"],
            yerr=sub_df["std_time"],
            fmt="none",
            ecolor="gray",
            elinewidth=1.5,
            capsize=3
        )

    plt.title(f"Computation Time vs. Sample Size for Matrix: {mat_name}", fontsize=16)
    plt.xlabel("Sample Size", fontsize=14)
    plt.ylabel("Mean Time (s)", fontsize=14)
    plt.xticks(rotation=45)
    plt.legend(title="Method")
    plt.tight_layout()
    
    filename = f"matrix_{mat_name.replace(' ', '_')}_time_plot.png"
    plt.savefig(os.path.join(output_dir, filename), dpi=300)
    plt.close()

print(f"Computation time plots saved to '{output_dir}/'")


Computation time plots saved to 'plots/'


In [38]:
df.head()


,matrix,sample_size,iteration,method,time,frobenius_norm,relative_error
0,Uniform Good,250,1,Leverage Sampling,0.054332,2.080475e+16,0.062989
1,Uniform Good,250,1,Sqrt Leverage Sampling,0.057373,2.142545e+16,0.064803
2,Uniform Good,250,1,Priority Sampling,0.052887,2.162665e+16,0.063835
3,Uniform Good,250,1,Threshold Sampling,0.061015,2.177947e+16,0.064301
4,Uniform Good,250,1,Uniform Sampling,0.004241,2.127004e+16,0.062498


In [39]:
import pandas as pd

# Assuming your dataframe is called df

# List of matrix types you are interested in
matrix_types = ["Uniform Good", "Uniform Bad", "Non-uniform Good", "Non-uniform Bad"]

# Create an empty dictionary to store the separate DataFrames
matrix_tables = {}

# Loop through each matrix type
for matrix_type in matrix_types:
    # Filter for this matrix type and sample size of 2000
    filtered_df = df[(df['matrix'] == matrix_type) & (df['sample_size'] == 2000)]
    
    # Group by 'method' and calculate the mean of 'relative_error' and 'time'
    grouped = filtered_df.groupby('method').agg({
        'relative_error': 'mean',
        'time': 'mean'
    }).reset_index()
    
    # Sort by relative_error ascending
    sorted_grouped = grouped.sort_values(by='relative_error', ascending=True).reset_index(drop=True)
    
    # Save to the dictionary
    matrix_tables[matrix_type] = sorted_grouped

# Now matrix_tables["Uniform Good"], matrix_tables["Uniform Bad"], etc. are your sorted tables


In [40]:
import pandas as pd

# Assuming your DataFrame is named df
matrix_types = ["Uniform Good", "Uniform Bad", "Non-uniform Good", "Non-uniform Bad"]
matrix_tables = {}

# Create separate tables
for matrix_type in matrix_types:
    filtered_df = df[(df['matrix'] == matrix_type) & (df['sample_size'] == 2000)]
    
    grouped = filtered_df.groupby('method').agg({
        'relative_error': 'mean',
        'time': 'mean'
    }).reset_index()
    
    sorted_grouped = grouped.sort_values(by='relative_error', ascending=True).reset_index(drop=True)
    matrix_tables[matrix_type] = sorted_grouped

# Now, generate LaTeX tables
for matrix_type, table in matrix_tables.items():
    print(f"\n\\textbf{{{matrix_type}}} Table:\n")
    print(table.to_latex(index=False, float_format="%.6f"))



\textbf{Uniform Good} Table:

\begin{tabular}{lrr}
\toprule
method & relative_error & time \\
\midrule
PHD JL (Hadamard) & 0.009969 & 0.174878 \\
Threshold Sampling & 0.021311 & 0.126851 \\
Priority Sampling & 0.021721 & 0.117339 \\
Gaussian JL & 0.022363 & 0.374924 \\
CountSketch JL & 0.022430 & 0.064586 \\
Uniform Sampling & 0.022690 & 0.025237 \\
Leverage Sampling & 0.024321 & 0.158274 \\
Sqrt Leverage Sampling & 0.024500 & 0.076609 \\
\bottomrule
\end{tabular}


\textbf{Uniform Bad} Table:

\begin{tabular}{lrr}
\toprule
method & relative_error & time \\
\midrule
Threshold Sampling & 0.021157 & 0.127315 \\
Priority Sampling & 0.021538 & 0.111438 \\
Gaussian JL & 0.022376 & 0.367146 \\
CountSketch JL & 0.022465 & 0.065128 \\
Uniform Sampling & 0.022831 & 0.032680 \\
Leverage Sampling & 0.024093 & 0.165971 \\
Sqrt Leverage Sampling & 0.024430 & 0.065521 \\
PHD JL (Hadamard) & 0.028490 & 0.168568 \\
\bottomrule
\end{tabular}


\textbf{Non-uniform Good} Table:

\begin{tabular}{lrr}
\to